NOTE: This notebook follows this sample notebook (with some modifications): https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing#scrollTo=IqM-T1RTzY6C


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
#ending = "_3e-4LR_10k_1b_test"  #change between different runs (if not want to continue from checkpoint!)
#ending = "_3e-4LR_R32_10k_1b_test"
#ending = "_3e-4LR_10k_3b_test"
#ending = "_3e-4LR_r32_10k_1b_val"
ending= "_3e-4LR_1b_val_BrainStoorming"

In [ ]:
from google.colab import drive
drive.mount("/content/drive") #newly added: force_remount=True

output_dir = f"/content/drive/MyDrive/unsloth_runs{ending}/llama32_3b_lora{ending}"

Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",  #"unsloth/Llama-3.2-3B-Instruct", #"unsloth/Llama-3.2-1B-Instruct", #"unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, #32,   #16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128    #DENNA KANSKE VI FAKTISKT VILL EXPERIMENTERA MED!  #16 was for most tests!  Lets test 32 here
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # 32, # 16,                                                            #and 32 here!  #was 16 for most tests.
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False, #True, #False,  # We support rank stabilized LoRA                #False was for most tests!  (perhaps lets keep it that way, stability seems ok i think)
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Data Prep
We now use the Llama-3.1 format for conversation style finetunes. We use Maxime Labonne's FineTome-100k dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format ("role", "content") instead of ("from", "value")/ Llama-3 renders multi turn conversations like below:

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
We use our get_chat_template function to get the correct chat template. We support zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3 and more.

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
from sklearn.model_selection import train_test_split
#dataset = load_dataset("mlabonne/FineTome-100k", split = "train") #original (full dataset)

def format_brainstoorming(examples):
    instruction = (examples.get("instruction") or "").strip()
    input_field = (examples.get("input") or "").strip()
    output= (examples.get("output") or "").strip()

    if input_field:
        user_input = instruction + " " + input_field
    else:
        user_input = instruction

    conv = [{"from": "human", "value": user_input}, {"from": "gpt",   "value": output}]
    return {"conversations": conv}

dataset = load_dataset("Wanfq/Explore_Instruct_Brainstorming_10k", split="train")
dataset = dataset.map(format_brainstoorming, remove_columns=dataset.column_names)  #match the format of fineTome!


README.md: 0.00B [00:00, ?B/s]

brainstorming_num-10000_depth-1_prune_fi(…):   0%|          | 0.00/12.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

We now use standardize_sharegpt to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:

{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
to

{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [ ]:
dataset[5]["conversations"]

[{'content': "Brainstorm potential themes for a company's upcoming online conference. Our company is hosting an online conference and we need ideas for potential themes to guide the presentations and discussions.",
  'role': 'user'},
 {'content': 'Here are five potential themes for the online conference:\n\n1. Innovation and forward-thinking in the industry\n2. Diversity, equity, and inclusion in the workplace\n3. The future of remote work and virtual collaboration\n4. E-commerce trends and strategies in a post-pandemic world\n5. Sustainability and ethical practices in business operations',
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

In [ ]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBrainstorm potential themes for a company's upcoming online conference. Our company is hosting an online conference and we need ideas for potential themes to guide the presentations and discussions.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere are five potential themes for the online conference:\n\n1. Innovation and forward-thinking in the industry\n2. Diversity, equity, and inclusion in the workplace\n3. The future of remote work and virtual collaboration\n4. E-commerce trends and strategies in a post-pandemic world\n5. Sustainability and ethical practices in business operations<|eot_id|>"

Train the model
Now let's use Huggingface TRL's SFTTrainer! More docs here: TRL SFT docs. We do 60 steps to speed things up, but you can set num_train_epochs=1 for a full run, and turn off max_steps=None. We also support TRL's DPOTrainer!

In [ ]:
split_set = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_set["train"]
val_and_eval = split_set["test"]

split_set_2 = val_and_eval.train_test_split(test_size=0.5, seed=42)
val_dataset = split_set_2["train"]
test_dataset = split_set_2["test"]
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

8000
1000
1000


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

"""trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use this for WandB etc
        save_steps = 20,
        save_total_limit = 3,
    ),
)"""


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset= val_dataset, #test_dataset,  #val_dataset,     #swap for test set when doing final eval after fine-tuning!
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        #learning_rate = 2e-4, #Base
        #learning_rate = 1e-4,  #second attempt
        learning_rate = 3e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        report_to = "none", # Use this for WandB etc
        save_strategy="steps",   #added now, not sure if necessary
        save_steps = 250,       #225 would have been better perhaps.
        save_total_limit = 3,

        #evaluation additions:
        eval_strategy = "steps",
        eval_steps = 250,     #225 would have been better perhaps.
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        #remove_unused_columns=False,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/8000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/1000 [00:00<?, ? examples/s]

We also use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=12):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=12):   0%|          | 0/1000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBrainstorm some ideas for a "day in the life" series that follows successful professionals in different industries. We want to create a series that showcases the daily routines and habits of successful professionals in various fields. Can you come up with some ideas for us?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere are ten potential ideas for a "day in the life" series:\n\n1. A day in the life of a CEO: Follow a CEO through their day and their habits and routines for staying productive and focused.\n2. A day in the life of a doctor: Follow a doctor through their day and their habits and routines for staying healthy and focused.\n3. A day in the life of a chef: Follow a chef through their day and their habits and routines for staying creative and inspired in the kitch

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                       Here are ten potential ideas for a "day in the life" series:\n\n1. A day in the life of a CEO: Follow a CEO through their day and their habits and routines for staying productive and focused.\n2. A day in the life of a doctor: Follow a doctor through their day and their habits and routines for staying healthy and focused.\n3. A day in the life of a chef: Follow a chef through their day and their habits and routines for staying creative and inspired in the kitchen.\n4. A day in the life of a musician: Follow a musician through their day and their habits and routines for staying productive and creative.\n5. A day in the life of a writer: Follow a writer through their day and their habits and routines for staying creative and productive.\n6. A day in the life of an athlete: Follow an athlete through their day and their habits and routines for staying physically and mentally fit.\n7. A day in the life o

We can see the System and Instruction prompts are successfully masked!

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.203 GB of memory reserved.


In [ ]:
next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
import os
from transformers.trainer_utils import get_last_checkpoint

trainer_stats = None
last_checkpoint = None
if os.path.isdir(output_dir):
  try:
    last_checkpoint = get_last_checkpoint(output_dir)
  except Exception:
    last_checkpoint = None

if last_checkpoint is not None:
  print(f"Continuing training from checkpoint: {last_checkpoint}")
  trainer_stats = trainer.train(resume_from_checkpoint=last_checkpoint)
else: #If not have checkpoint, just train as usual.
  print("Training from scratch!")
  trainer_stats = trainer.train()

Training from scratch!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 1 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
250,1.501100,1.499958
500,1.510200,1.477387
750,1.483200,1.463119
1000,1.502300,1.455924


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
#Same eval set as in during training (so we swap to holdout testset for final eval!)
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.4559260606765747, 'eval_runtime': 68.4712, 'eval_samples_per_second': 14.605, 'eval_steps_per_second': 7.302, 'epoch': 1.0}


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1576.7866 seconds used for training.
26.28 minutes used for training.
Peak reserved memory = 2.182 GB.
Peak reserved memory for training = 0.979 GB.
Peak reserved memory % of max memory = 14.802 %.
Peak reserved memory for training % of max memory = 6.641 %.


Inference
Let's run the model! You can change the instruction and input - leave the output blank!

[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct here

We use min_p = 0.1 and temperature = 1.5. Read this Tweet for more information on why. (https://x.com/menhguin/status/1826132708508213629)

You can also use a TextStreamer for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here is the continued fibonacci sequence: 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1594, 2584, 4181, 6765, 10946, 17711, 28657, 465369, 750991, 1212905, 1962883, 3178110, 5148414, 8253017, 13336920, 21417161, 357


In [ ]:
model.save_pretrained(f"lora_model{ending}") # Local saving
tokenizer.save_pretrained(f"lora_model{ending}")
model.push_to_hub(f"ViktorMardskog/NON_GGUF_lora_model{ending}", token = "your token") # Online saving
tokenizer.push_to_hub(f"ViktorMardskog/NON_GGUF_lora_model{ending}", token = "your token") # Online saving

FastLanguageModel.for_inference(model)
model.save_pretrained_gguf(f"lora_model{ending}", tokenizer, quantization_method = "q4_k_m")
model.push_to_hub_gguf(f"ViktorMardskog/lora_model{ending}", tokenizer, quantization_method = "q4_k_m", token = "your token")



README.md:   0%|          | 0.00/619 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 28.9kB / 22.6MB            

Saved model to https://huggingface.co/ViktorMardskog/NON_GGUF_lora_model_3e-4LR_1b_val_BrainStoorming


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...2-1b-instruct.Q4_K_M.gguf:   0%|          |  559kB /  808MB            

Uploading config.json...
Uploading Ollama Modelfile...
Unsloth: Successfully uploaded GGUF to https://huggingface.co/ViktorMardskog/lora_model_3e-4LR_1b_val_BrainStoorming
Unsloth: Cleaning up temporary files...


'ViktorMardskog/lora_model_3e-4LR_1b_val_BrainStoorming'